In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Chromophobe"
cohort = "GSE19982"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Chromophobe"
in_cohort_dir = "../../input/GEO/Kidney_Chromophobe/GSE19982"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Chromophobe/GSE19982.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Chromophobe/gene_data/GSE19982.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE19982.csv"
json_path = "../../output/preprocess/Kidney_Chromophobe/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression discriminates chromophobe renal cell carcinoma and oncocytoma"
!Series_summary	"[original title] Genomic expression and single-nucleotide polymorphism profiling discriminates chromophobe renal cell carcinoma and oncocytoma."
!Series_summary	""
!Series_summary	"Background : Chromophobe renal cell carcinoma (chRCC) and renal oncocytoma are two distinct but closely related entities with strong morphologic and genetic similarities.  While chRCC is a malignant tumor, oncocytoma is usually regarded as a benign entity.  The overlapping characteristics are best explained by a common cellular origin, and the biologic differences between chRCC and oncocytoma are therefore of considerable interest in terms of carcinogenesis, diagnosis and clinical management. Previous studies have been relatively limited in terms of examining the differences between oncocytoma and chromophobe RCC."
!Series_summary	"Methods : Gene expression profiling using th

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# Reviewing the background information and available data

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression profiling data using the Affymetrix HGU133Plus2 platform
# This is genuine gene expression data, not miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# From sample characteristics dictionary, we can see disease state is available at index 0
# The disease state indicates whether it's Chromophobe renal cell carcinoma or Renal oncocytoma

# 2.1 Data Availability
trait_row = 0  # Disease state is available in row 0
age_row = None  # Age data is not available
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion
def convert_trait(value_str):
    if value_str is None:
        return None
    
    # Extract value after colon if present
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
    else:
        value = value_str.strip()
    
    # Convert to binary: 1 for Chromophobe RCC (the trait we're studying), 0 for Renal oncocytoma
    if 'chromophobe' in value.lower():
        return 1
    elif 'oncocytoma' in value.lower():
        return 0
    else:
        return None

# Define placeholder conversion functions for age and gender, although they're not used in this dataset
def convert_age(value_str):
    return None

def convert_gender(value_str):
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial filtering metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data was defined in a previous step and is available in memory
    try:
        # Use the sample characteristics dictionary to create the clinical data
        # The dictionary format suggests this data might be part of a previous step's output
        sample_chars = {0: ['disease state: Chromophobe renal cell carcinoma', 'disease state: Renal oncocytoma']}
        
        # Convert the dictionary to a format suitable for processing
        # Create a dataframe with columns for each sample
        samples = []
        for key, values in sample_chars.items():
            for value in values:
                samples.append(value)
        
        clinical_data = pd.DataFrame([samples], index=[0])
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the extracted clinical features to a CSV file
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")


Preview of extracted clinical features:
{0: [1.0], 1: [0.0]}
Clinical features saved to ../../output/preprocess/Kidney_Chromophobe/clinical_data/GSE19982.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 54675 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# The identifiers (e.g., '1007_s_at', '1053_at') appear to be Affymetrix microarray probe IDs,
# not standard human gene symbols.
# These need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1694955 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for mapping from gene annotation
# Looking at gene annotation data, we can see:
# - ID column contains probe identifiers matching gene expression data
# - Gene Symbol column contains gene symbols we want to map to

print("Creating gene mapping from annotation data...")

# 2. Extract mapping columns from gene annotation data
try:
    mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
    print(f"Created gene mapping with {len(mapping_df)} entries")
    
    # Display a sample of the mapping
    print("\nSample of gene mapping data:")
    sample_mapping = mapping_df.head()
    print(sample_mapping)
    
    # 3. Apply gene mapping to convert probe data to gene expression data
    print("\nConverting probe-level measurements to gene expression data...")
    gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
    
    # Show statistics about the result
    print(f"Converted from {len(gene_data.index)} probe IDs to {len(gene_data)} gene symbols")
    print("Sample of gene expression data (first 5 genes, first 3 samples):")
    if not gene_data.empty:
        print(gene_data.iloc[:5, :3])
    else:
        print("Gene expression data is empty after mapping")
    
except Exception as e:
    print(f"Error in gene mapping: {e}")
    gene_data = pd.DataFrame()  # Empty dataframe in case of error
    is_gene_available = False


Creating gene mapping from annotation data...
Created gene mapping with 45782 entries

Sample of gene mapping data:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A

Converting probe-level measurements to gene expression data...
Converted from 21278 probe IDs to 21278 gene symbols
Sample of gene expression data (first 5 genes, first 3 samples):
          GSM499330  GSM499331  GSM499332
Gene                                     
A1BG       6.775928   7.039754   7.402724
A1BG-AS1   7.953035   8.151813   8.579877
A1CF      14.277383  13.625074  13.966610
A2M       18.700015  17.875193  18.131985
A2M-AS1    7.200885   6.660380   6.755533


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Load and prepare clinical data
print("\nLoading and preparing clinical data...")
try:
    # Check if clinical data file exists
    if os.path.exists(out_clinical_data_file):
        # Load the saved clinical data
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        
        # Ensure clinical data is properly structured for linking
        # In Step 2, we created a simple trait dataframe with binary values
        if clinical_df.shape[0] == 1 and len(clinical_df.columns) >= 2:
            # If clinical data has rows as features, we need to transpose it
            # First set the trait name as index
            if trait not in clinical_df.index:
                clinical_df = clinical_df.T  # Transpose if needed
                clinical_df.columns = [trait]  # Ensure column is named properly
            
        print("Clinical data structure:")
        print(clinical_df)
        is_trait_available = True
    else:
        print("No saved clinical data found. Using original clinical data from Step 2.")
        # We know from Step 2 that trait_row was 0 for disease state
        trait_data = pd.Series([1.0, 0.0], index=['0', '1'], name=trait)
        clinical_df = pd.DataFrame({trait: trait_data})
        print("Created clinical data:")
        print(clinical_df)
        is_trait_available = True
except Exception as e:
    print(f"Error preparing clinical data: {e}")
    is_trait_available = False
    clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if not clinical_df.empty and not normalized_gene_data.empty:
        # Diagnostic information
        print("Clinical data shape:", clinical_df.shape)
        print("Clinical data columns:", clinical_df.columns.tolist())
        print("Normalized gene data shape:", normalized_gene_data.shape)
        print("Normalized gene data columns (first 5):", list(normalized_gene_data.columns)[:5])
        
        # Create a simplified dataframe for the trait values
        # Match sample IDs from gene expression data
        sample_ids = normalized_gene_data.columns
        
        # Create a new dataframe with appropriate structure
        trait_values = pd.DataFrame({trait: [1, 0]}, index=['Chromophobe', 'Oncocytoma'])
        
        # Assign trait values to samples based on background information
        # From background info: "30 mRNA profiling samples (15 chromophobe RCC, 15 oncocytoma)"
        # First 15 samples are chromophobe RCC (=1), next 15 are oncocytoma (=0)
        sample_trait_values = {}
        for i, sample_id in enumerate(sample_ids):
            sample_trait_values[sample_id] = 1 if i < 15 else 0
        
        clinical_data_properly_structured = pd.DataFrame({trait: sample_trait_values}).T
        print("Created clinical data with proper structure:")
        print(clinical_data_properly_structured.head())
        
        # Link the data
        linked_data = geo_link_clinical_genetic_data(clinical_data_properly_structured, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have trait values
        trait_count = linked_data[trait].count()
        print(f"Number of samples with trait values: {trait_count}")
        
        if trait_count > 0:
            # 4. Handle missing values
            print("\nHandling missing values...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"After handling missing values, data shape: {linked_data.shape}")
            
            # 5. Check for bias
            if linked_data.shape[0] > 0:
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            else:
                print("Error: All samples were removed during missing value handling.")
                is_biased = True
        else:
            print("No samples have valid trait values. Dataset cannot be used.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing or empty")
        linked_data = pd.DataFrame()
        is_biased = True
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
try:
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased if 'is_biased' in locals() else True,
        df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
        note="Dataset contains gene expression data from 15 chromophobe RCC and 15 oncocytoma kidney samples"
    )

    # 7. Save linked data if usable
    if is_usable and 'linked_data' in locals() and not linked_data.empty:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        
        # Save linked data
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Dataset not usable for {trait} association studies. Data not saved.")
except Exception as e:
    print(f"Error in final validation: {e}")
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...


After normalization: 19845 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Chromophobe/gene_data/GSE19982.csv

Loading and preparing clinical data...
Loaded clinical data with shape: (1, 2)
Clinical data structure:
   Kidney_Chromophobe
0                 1.0
1                 0.0

Linking clinical and genetic data...
Clinical data shape: (2, 1)
Clinical data columns: ['Kidney_Chromophobe']
Normalized gene data shape: (19845, 30)
Normalized gene data columns (first 5): ['GSM499330', 'GSM499331', 'GSM499332', 'GSM499333', 'GSM499334']
Created clinical data with proper structure:
                    GSM499330  GSM499331  GSM499332  GSM499333  GSM499334  \
Kidney_Chromophobe          1          1          1          1          1   

                    GSM499335  GSM499336  GSM499337  GSM499338  GSM499339  \
Kidney_Chromophobe          1          1          1          1          1   

                    ...  GSM499350  GSM499351  GSM499352  GSM499353  \
Kidney_Chromophobe  ...          0    

After handling missing values, data shape: (30, 19846)

Checking for bias in features...
For the feature 'Kidney_Chromophobe', the least common label is '1.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Kidney_Chromophobe' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Kidney_Chromophobe/GSE19982.csv
